<center>
    
# 利用LDA算法进行主题词提取

<br>
</center>

<img src='../static/img/logo.jpg' align='right' style="width:260px;height:60px;display:block"/>

In [1]:
!pip install pyecharts==0.5.11 && pip install baidu-aip

In [14]:
import pandas as pd
import numpy as np
import warnings
import re
from sklearn.manifold import TSNE
import jieba
import jieba.analyse
import matplotlib.pyplot as plt
from pyecharts import WordCloud
from collections import Counter
import multiprocessing
from gensim.models import Word2Vec
from aip import AipNlp
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel

# 忽略警告
warnings.filterwarnings('ignore')

In [15]:
# 查看数据的基本格式
df = pd.read_csv('./data/JianShuJiaoYou-All-Data.csv',encoding='utf-8')
print(df.columns)
# df.head(5)

Index(['Page', 'Num', 'Author', 'Author_Url', 'Title', 'Time', 'Read',
       'Comment', 'Like', 'Abstract', 'Artical_Url', 'Image_Url',
       'Artical_Content'],
      dtype='object')


### 提取文章中200个关键词绘制词云

In [16]:
content = df['Artical_Content'].tolist()
# print(len(content),'\n',content[-1])

In [17]:
# 载入停用词
# quoting : int or csv.QUOTE_* instance, default 0
# 控制csv中的引号常量。可选 QUOTE_MINIMAL (0), QUOTE_ALL (1), QUOTE_NONNUMERIC (2) or QUOTE_NONE (3)
 
stopwords = pd.read_csv('./data/chinesestopword.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
# 默认输出5个
stopwords.head()

,stopwords
0,!
1,""""
2,#
3,$
4,%


In [18]:
# 结巴分词
pattern = re.compile('\d+')

# 存放词语和词频
wordsCounter = Counter()

for line in content:
    segs = jieba.lcut(line)
    for seg in segs:
        if len(seg)>1 and seg != '\r\n' and re.search(pattern,seg)==None:
            wordsCounter[seg] += 1

print(wordsCounter.most_common(100))

# 将Counter的键提取出来做list
segment = list(wordsCounter)

# 将分好的词列表转化为词典
words = pd.DataFrame({'segment':segment})

# 剔除停用词
words = words[~words['segment'].isin(stopwords['stopwords'])]

[('喜欢', 9535), ('一个', 9314), ('自己', 9078), ('我们', 7539), ('没有', 4949), ('可以', 4830), ('时候', 3910), ('知道', 3571), ('一起', 3481), ('因为', 3445), ('就是', 3358), ('什么', 3132), ('简书', 2948), ('生活', 2787), ('希望', 2735), ('还是', 2734), ('觉得', 2636), ('朋友', 2621), ('这个', 2549), ('不是', 2468), ('如果', 2450), ('现在', 2365), ('很多', 2363), ('所以', 2363), ('但是', 2321), ('这样', 2155), ('不会', 2069), ('文章', 1981), ('一直', 1926), ('开始', 1744), ('真的', 1697), ('那么', 1672), ('一样', 1622), ('时间', 1606), ('可能', 1567), ('他们', 1543), ('只是', 1539), ('看到', 1539), ('还有', 1507), ('其实', 1505), ('大家', 1490), ('故事', 1452), ('文字', 1448), ('工作', 1440), ('感觉', 1368), ('一些', 1360), ('别人', 1355), ('那个', 1344), ('一定', 1326), ('已经', 1290), ('事情', 1283), ('然后', 1267), ('我会', 1264), ('大学', 1231), ('世界', 1229), ('遇见', 1222), ('虽然', 1216), ('特别', 1189), ('爱情', 1166), ('不要', 1155), ('写作', 1151), ('东西', 1122), ('怎么', 1107), ('地方', 1103), ('一次', 1096), ('认识', 1082), ('这么', 1073), ('后来', 1062), ('这里', 1062), ('努力', 1060), ('可是', 1055), ('一下'

### word2vec模型计算词的相似度

In [19]:
# 基于TF-IDF算法的关键词抽取
contentStr = ''.join(content)
# keyWordsList = jieba.analyse.extract_tags(contentStr,topK=200,allowPOS=('ns','n'))
# textrank1 = ' '.join(keyWordsList)
# textrank1

In [20]:
# 利用word2vec找出关联词语，将语料整理成嵌套列表的形式
corpus = []

# 将停用词dataFrame转化为列表
stopwordsList = stopwords['stopwords'].tolist()
# print(stopwordsList[:10])
for line in content:
    segs = jieba.lcut(line)
    segs = filter(lambda x:len(x)>1, segs)
    segs = filter(lambda x:re.search(pattern,x)==None, segs)
    segs = filter(lambda x:x not in stopwordsList, segs)
    
    corpus.append(list(segs))

In [21]:
# for article in corpus:
dictionary = corpora.Dictionary(corpus)
# 将 dictionary 转化为一个词袋
common_corpus = [dictionary.doc2bow(text) for text in corpus]
tfidf = models.TfidfModel(common_corpus)
corpusTfidf = tfidf[common_corpus]

lda = LdaModel(corpusTfidf, num_topics=10, id2word = dictionary, passes=20)

results = lda.print_topics(num_topics=10, num_words=3)
for res in results:
    print(res[1])

0.000*"傻大闹" + 0.000*"宝莱坞" + 0.000*"阿米尔"
0.000*"饭堂" + 0.000*"菜谱" + 0.000*"潮头"
0.000*"自已" + 0.000*"上心" + 0.000*"hong"
0.000*"胶囊" + 0.000*"兵法" + 0.000*"七秒"
0.001*"婆婆" + 0.000*"郑州" + 0.000*"军训"
0.000*"沐阳" + 0.000*"第三句" + 0.000*"后会有期"
0.000*"小叔叔" + 0.000*"小贤有" + 0.000*"蔷薇"
0.000*"最酷" + 0.000*"桔子" + 0.000*"盼望"
0.000*"打一" + 0.000*"热评" + 0.000*"狗蛋"
0.002*"喜欢" + 0.002*"树洞" + 0.002*"投稿"


In [22]:
# 将LDA模型保存起来方便日后对新文章进行主题预测
lda.save('lda.model')
test = "生活是一座围城，但校园不是。校园是时光精心雕刻的礼物，里面到处有十几岁长发飘飘的女孩子，朝气执着的男孩子，有单纯追求的理想和生活，还有那些用心守候的友情爱情。秋天的梧桐树铺满教学楼的道路，在密密麻麻的习题中痴迷地偷看窗外的世界，静谧、自由；冬天的大雪来势汹汹，裹着厚棉袄走在冰封的湖边独自去上课，长大的世界只有自己可以取暖；春天，随着冰雪一起褪去的还有沉重的身心，但新的一年生活好像也没什么变化；夏天有吃不完的西瓜和冰棍，如果你可以和我一起逃课，我会告诉你后山夜晚的星星很美。提起校园，或许它并不精彩，重复到近乎单调，但我们都会无比怀念年轻的自己和那些有梦做的年纪。春天濛濛的细雨还没有离开大地，夏日离别的笙歌已在悄悄响起。又快到一年毕业季，「简书交友」专题携手「简书校园」、「简书会员」、「摄影」一起推出「恰同学少年」校园创作交友大赛。请将文章以以下形式投稿至「」专题、摄影作品可投稿至「」专题。本次活动设置了丰富的奖项和奖品，参赛内容可为小学到大学，培训学校等等你经历过的任何一个校园，校园必须为的大学，虚拟意义上的校园无法参赛，，所有简友仅需写出你曾经经历过的校园生活即可参赛。感谢对本次校园创作交友大赛的，除了提供奖金支持以外，还带来了八折的购买折扣。已是简书会员的简友参与本次比赛即可线上时长；所有参与本次比赛的简友均可获得购买简书会员的资格。详细会员权益请见：参加摄影奖的文章不参考文章质量，仅参考摄影作品，但建议对摄影作品加上一定的文字描述。片子必须原创，拒绝糊片，张数不限，可用snapseed、PS等软件修图，一旦发现盗图，永久取消参与简书活动的资格；一等奖一名：500元二等奖三名：200元三等奖五名：50元创作奖视文字部分内容而定，摄影仅做锦上添花的辅助说明。一等奖二名：1000元二等奖三名：500元三等奖五名：200元凡在本次比赛参与作品中找到同校同学，你们可以将自己的互相发布到彼此的文章，只要有五名以上的同校同学，你可以获得19元打赏。每人仅能获得一次且必须在自己的文章评论区集齐五名校友评论。收集齐后点击链接附上你的文章链接和校友评论楼层，通过验证即可获奖。登记地址：交友专题会对你提供的文章进行质量审核，单纯为了凑齐人数而没达到参赛标准的文章将被视为不符合规则。本次比赛人气最高的五篇文章和点赞人气前三的评论作者都可以获得人气奖，人气奖可与以上奖项重复获得。文章评分：0.4阅读量+0.3 评论数+0.3 点赞数，得分取前5名。人气奖奖品为：100元2018年3月27日-2018年4月20日，预计2018年5月上旬公布结果，获奖结果首发（微信号：jianshuio）。1.文章建议采用记叙性、回忆性散文、诗歌，校园生活和校园情，请勿全文叙述你的爱情（同理友情、师生情等等），不接受小说等虚构类作品；2.快速找到校友：点击列表找到自己的学校，或者点击联系负责人加入学校社群。结交校友可以100%获得。3.活动不局限于在校生参与，任何简友都可以写出你憧憬/经历过的校园生活；4.简书对所有参赛文章具有使用权，简书交友专题对活动具有最终解释权；「恰同学少年」校园创作交友大赛同步合作伙伴、引力说（微信号：GravityYLS）、清华帮（微信号：THU_bang）、小也电台（微信号：xiaoyeradio），获奖作品将同步发表在以上平台。加入简书第一步，添加简书交友官方微信群（已在前面九群的请不要重复添加）。"

test = jieba.lcut(test)

# 文档转换成bow
doc_bow = dictionary.doc2bow(test)
# 得到新文档的主题分布
doc_lda = lda[doc_bow]
type = doc_lda[1][0]
print(results[type][1])

0.002*"喜欢" + 0.002*"树洞" + 0.002*"投稿"
